### Imports

In [1]:
from __future__ import annotations

from pathlib import Path

from kub.course.plotlib.simulationPlotFactory import SimulationPlotFactory
from kub.course.simlib.simulation import FMUSimulation

# Find repository root: check if database/ exists at current location
repo_root = (
    Path.cwd() if (Path.cwd() / "database").exists() else Path.cwd().parent.parent
)
databasePath = repo_root / "database" / "day1" / "FMUs"

### **Exercise 5: Solar Radiation**

**<ins>Question 1:<ins>**

**Analyze the Shortwave Radiation data for the four vertical walls (Surfaces 1, 2, 3, and 4) over a full day.** \
Based only on the observed radiation patterns (when the radiation is highest and lowest), **identify and label which surface corresponds to the NORTH, SOUTH, EAST, and WEST orientations.**

**<ins>Question 2:<ins>**

**Focus on the wall you identified as SOUTH.** \
Systematically **modify its Tilt Angle β between 180° (horizontal) and 90° (vertical) by 10° increment.** \
What specific tilt angle **maximizes the total daily solar energy received** (or the maximum instantaneous Shortwave Radiation) on this South-facing surface?

**<ins>Question 3:<ins>**

**After setting the wall 1 Azimuth to 90°, you may observe that its power curve (in W) is similar to the wall 2 curve, but vertically offset.** \
**Explain this offset** and the observed curve similarity. \
Knowing that the absorptivity of all walls is 0.6 **what must be the numerical difference between Wall 1 and Wall 2 to cause this difference in total power (W) ?** (give a rough estimation)

**<ins>Question 4:<ins>**

Modify Azimuth between 0° and 359° and observe change.


In [20]:
sim1 = FMUSimulation(databasePath / "Exercises_SolarRadiation_5.fmu")

sim1.initialize(startTime=0.0, stopTime=86400 * 4, timeStep=3600.0)
sim1.initParameters(
    ## default: azi=0, tilt=90
    ## tilt:
    ### 0° : face the sky
    ### 180° : face the ground

    {"building.wall_SOUTH.angleDegAzi": 90.0, "building.wall_SOUTH.angleDegTil": 90.0}
)
sim1.exitInitialization()

outputNames = [
    "building.surfacesToAmbience.toSurfacesPorts[1].heatPortSw.Q_flow",
    "building.surfacesToAmbience.toSurfacesPorts[2].heatPortSw.Q_flow",
    "building.surfacesToAmbience.toSurfacesPorts[3].heatPortSw.Q_flow",
    "building.surfacesToAmbience.toSurfacesPorts[4].heatPortSw.Q_flow",
]
data1 = sim1.run(outputNames)
sim1.terminate()

sim1Factory = SimulationPlotFactory()

sim1Factory.plot_multi_curves(
    time=data1["time"],
    data_type="heat_flux",
    data_dict={
        "Surface 1": data1[
            "building.surfacesToAmbience.toSurfacesPorts[1].heatPortSw.Q_flow"
        ],
        "Surface 2": data1[
            "building.surfacesToAmbience.toSurfacesPorts[2].heatPortSw.Q_flow"
        ],
        "Surface 3": data1[
            "building.surfacesToAmbience.toSurfacesPorts[3].heatPortSw.Q_flow"
        ],
        "Surface 4": data1[
            "building.surfacesToAmbience.toSurfacesPorts[4].heatPortSw.Q_flow"
        ],
    },
    title="Exercise 5 — Short wave radiation (Direct + Diffuse)",
);

In [23]:
## calc difference
w1 = data1["building.surfacesToAmbience.toSurfacesPorts[1].heatPortSw.Q_flow"]
w2 = data1["building.surfacesToAmbience.toSurfacesPorts[2].heatPortSw.Q_flow"]

import numpy as np
ratio = w1/w2
ratio = ratio[~np.isnan(ratio)]
r = np.average(ratio)

res = np.arctan(r) * 180/np.pi;
print("angle between walls?", res)

angle between walls? 30.650667957052857


/tmp/ipykernel_42900/74805021.py:6: RuntimeWarning:

invalid value encountered in divide



### **Exercise 6: Isotropic vs Anisotropic**

**<ins>Question 1:<ins>**

Observe and comment on short wave radiation for south wall displayed for both models.

**<ins>Question 2:<ins>**

Modify **Azimuth** for both model between **0°** and **270°** and **observe change in diffuse radiation for Perez model**.

In [28]:
# to modify
## default: 0
input_angleDegAziWall = 270

# Isotropic model
simRadIsotropic = FMUSimulation(databasePath / "Exercises_SolarRadiation_5.fmu")
simRadIsotropic.initialize(startTime=0.0, stopTime=86400.0, timeStep=3600.0)
simRadIsotropic.initParameters(
    {
        "building.wall_SOUTH.angleDegAzi": input_angleDegAziWall,
        "building.wall_SOUTH.angleDegTil": 90.0,
    }
)
simRadIsotropic.exitInitialization()

# Perez model
simRadPerez = FMUSimulation(databasePath / "Exercises_SolarRadiation_6.fmu")

simRadPerez.initialize(startTime=0.0, stopTime=86400.0, timeStep=3600.0)
simRadPerez.initParameters(
    {
        "building.wall_SOUTH.angleDegAzi": input_angleDegAziWall,
        "building.wall_SOUTH.angleDegTil": 90.0,
    }
)
simRadPerez.exitInitialization()

In [29]:
# Simulation and plots
outputNames = [
    "building.surfacesToAmbience.toSurfacesPorts[1].heatPortSw.Q_flow",
    "building.surfacesToAmbience.toSurfacesPorts[1].radiationPort_in.IrrDir",
    "building.surfacesToAmbience.toSurfacesPorts[1].radiationPort_in.IrrDif",
]

dataIso = simRadIsotropic.run(outputNames)
dataPer = simRadPerez.run(outputNames)

simRadIsotropic.terminate()
simRadPerez.terminate()

simFactory = SimulationPlotFactory()

simFactory.plot_multi_curves(
    time=dataIso["time"],
    data_type="heat_flux",
    data_dict={
        "Short wave radiation Isotropic": dataIso[
            "building.surfacesToAmbience.toSurfacesPorts[1].heatPortSw.Q_flow"
        ],
        "Short wave radiation Perez": dataPer[
            "building.surfacesToAmbience.toSurfacesPorts[1].heatPortSw.Q_flow"
        ],
    },
    title="Exercise 6 — Short wave radiation (Direct + Diffuse)",
)

simFactory.plot_multi_curves(
    time=dataIso["time"],
    data_type="surface_heat_flux",
    data_dict={
        "Direct radiation Isotropic": dataIso[
            "building.surfacesToAmbience.toSurfacesPorts[1].radiationPort_in.IrrDir"
        ],
        "Diffuse radiation Isotropic": dataIso[
            "building.surfacesToAmbience.toSurfacesPorts[1].radiationPort_in.IrrDif"
        ],
        "Direct radiation Perez": dataPer[
            "building.surfacesToAmbience.toSurfacesPorts[1].radiationPort_in.IrrDir"
        ],
        "Diffuse radiation Perez": dataPer[
            "building.surfacesToAmbience.toSurfacesPorts[1].radiationPort_in.IrrDif"
        ],
    },
    title="Exercise 6 — Solar radiation (Direct + Diffuse)",
);